In [19]:
import pandas as pd

org_df = pd.read_csv('parse_tree.csv')
org_df = org_df.set_index(org_df['token_idx'])

org_df

,head_token_idx,label,pos,text,token_idx
token_idx,,,,,
0,7,ADVMOD,PRON,언제,0
1,2,NN,NOUN,주문,1
2,7,NSUBJ,UNKNOWN,했는지,2
3,7,ADVMOD,ADV,잘,3
4,7,NSUBJ,NOUN,기억,4
5,4,PRT,PRT,은,5
6,7,NEG,ADV,안,6
7,10,ADVCL,VERB,나는데,7
8,10,ADVMOD,ADV,아무튼,8


In [20]:
l = [[14, 10, 7, 0], [14, 10, 7, 2, 1], [14, 10, 7, 3], [14, 10, 7, 4, 5], [14, 10, 7, 6], [14, 10, 8], [14, 10, 9], [14, 10, 11], [14, 12], [14, 13]]

df = pd.DataFrame(l).fillna(-1)
df = df.astype(int)
display(df)

for ll in l:
    display(org_df.loc[ll])

,0,1,2,3,4
0,14,10,7,0,-1
1,14,10,7,2,1
2,14,10,7,3,-1
3,14,10,7,4,5
4,14,10,7,6,-1
5,14,10,8,-1,-1
6,14,10,9,-1,-1
7,14,10,11,-1,-1
8,14,12,-1,-1,-1
9,14,13,-1,-1,-1


,head_token_idx,label,pos,text,token_idx
token_idx,,,,,
14,14,ROOT,VERB,주세요,14
10,14,ADVCL,UNKNOWN,할꺼,10
7,10,ADVCL,VERB,나는데,7
0,7,ADVMOD,PRON,언제,0


,head_token_idx,label,pos,text,token_idx
token_idx,,,,,
14,14,ROOT,VERB,주세요,14
10,14,ADVCL,UNKNOWN,할꺼,10
7,10,ADVCL,VERB,나는데,7
2,7,NSUBJ,UNKNOWN,했는지,2
1,2,NN,NOUN,주문,1


,head_token_idx,label,pos,text,token_idx
token_idx,,,,,
14,14,ROOT,VERB,주세요,14
10,14,ADVCL,UNKNOWN,할꺼,10
7,10,ADVCL,VERB,나는데,7
3,7,ADVMOD,ADV,잘,3


,head_token_idx,label,pos,text,token_idx
token_idx,,,,,
14,14,ROOT,VERB,주세요,14
10,14,ADVCL,UNKNOWN,할꺼,10
7,10,ADVCL,VERB,나는데,7
4,7,NSUBJ,NOUN,기억,4
5,4,PRT,PRT,은,5


...

,head_token_idx,label,pos,text,token_idx
token_idx,,,,,
14,14,ROOT,VERB,주세요,14
10,14,ADVCL,UNKNOWN,할꺼,10
8,10,ADVMOD,ADV,아무튼,8


,head_token_idx,label,pos,text,token_idx
token_idx,,,,,
14,14,ROOT,VERB,주세요,14
10,14,ADVCL,UNKNOWN,할꺼,10
9,10,NN,NOUN,취소,9


,head_token_idx,label,pos,text,token_idx
token_idx,,,,,
14,14,ROOT,VERB,주세요,14
10,14,ADVCL,UNKNOWN,할꺼,10
11,10,PRT,UNKNOWN,니까,11


,head_token_idx,label,pos,text,token_idx
token_idx,,,,,
14,14,ROOT,VERB,주세요,14
12,14,ADVMOD,ADJ,그렇게,12


,head_token_idx,label,pos,text,token_idx
token_idx,,,,,
14,14,ROOT,VERB,주세요,14
13,14,AUX,UNKNOWN,해,13


In [21]:
import numpy as np

def get_diff_position(df, idx1, idx2):
    diff_p = 0
    _, cols = df.shape
    
    for c_idx in range(cols-1, -1, -1): 
        if df[c_idx][idx1:idx1+1].values != df[c_idx][idx2:idx2+1].values:
            diff_p = c_idx
            
    return diff_p

def get_dependency(df, depth):
    backwards = False
    idxs = []
    for index, row in df.iterrows():
        if backwards:
            idxs.insert(position,row['token_idx'])
        else:
            idxs.append(row['token_idx'])
            
        backwards, position = (True, len(idxs) - 1) if row['label'] in ['SUFF','PTR'] else (False, None)
        
        if row['label'] in ['DOBJ','NSUBJ']:
            es_dict['what']['depth'] = depth
            es_dict['what']['label'] = row['label']
            es_dict['what']['tokens'] = idxs[:]
        elif row['label'] in ['ADVCL']:
            es_dict['why']['depth'] = depth
            es_dict['why']['label'] = row['label']
            es_dict['why']['tokens'] = idxs[:]
        elif row['label'] == 'ROOT':
            es_dict['how']['depth'] = depth
            es_dict['how']['label'] = row['label']
            es_dict['how']['tokens'] = idxs[:]

    return idxs

    
rows, cols = df.shape
prev_dp = cols

es_dict = {
    "when":{
        'depth':None,
        'label':None,
        'tokens':[]
    },
    "what":{
        'depth':None,
        'label':None,
        'tokens':[]
    },
    "how":{
        'depth':None,
        'label':None,
        'tokens':[]
    },
    "why":{
        'depth':None,
        'label':None,
        'tokens':[]
    }
}

for r_idx in range(rows):
    dp = get_diff_position(df, r_idx, r_idx+1)
    cul_cols = [i for i in range(dp,cols)]
    print("diff:",dp)
    
    t = df[cul_cols][r_idx:r_idx+1].values[0][::-1]
    t = t[np.where(t>-1)]
    print(t)
    print(org_df.loc[t])
    get_dependency(org_df.loc[t], dp)
    
    prev_dp = dp
    
display(es_dict)

diff: 3
[0]
           head_token_idx   label   pos text  token_idx
token_idx                                              
0                       7  ADVMOD  PRON   언제          0
diff: 3
[1 2]
           head_token_idx  label      pos text  token_idx
token_idx                                                
1                       2     NN     NOUN   주문          1
2                       7  NSUBJ  UNKNOWN  했는지          2
diff: 3
[3]
           head_token_idx   label  pos text  token_idx
token_idx                                             
3                       7  ADVMOD  ADV    잘          3
diff: 3
[5 4]
           head_token_idx  label   pos text  token_idx
token_idx                                             
5                       4    PRT   PRT    은          5
4                       7  NSUBJ  NOUN   기억          4
diff: 2
[6 7]
           head_token_idx  label   pos text  token_idx
token_idx                                             
6                       7    NEG   ADV 

{'how': {'depth': 0, 'label': 'ROOT', 'tokens': [13, 14]},
 'what': {'depth': 3, 'label': 'NSUBJ', 'tokens': [5, 4]},
 'when': {'depth': None, 'label': None, 'tokens': []},
 'why': {'depth': 1, 'label': 'ADVCL', 'tokens': [11, 10]}}

In [22]:
es_df = pd.read_csv('es_df.csv')
display(es_df)

,kind,obj_entities,mind_entities,obj_means,mind_means,action,branch,morphs,origin
0,how,[],[],[],[],[],"[12, 14, 13]","['그렇게/Adverb', '줄다/Verb', '해/Noun']",그렇게 주세요 해
1,what,[],[],[],[],[],"[4, 5]","['기억/Noun', '은/Josa']",기억은
2,why,['취소/Noun'],[],['cancel'],[],"['하다', '나다']","[0, 3, 7, 6, 8, 9, 10, 11]","['언제/Noun', '자다/Verb', '나다/Verb', '안/Noun', '아...",언제 잘 나는데 안 아무튼 취소 할꺼니까


In [12]:
branchs = [[5, 2, 0, 1], [5, 2, 3], [5, 2, 4], [5, 6], [5, 7]]
for branch in branchs:
    print(org_df[org_df['token_idx'] == branch[-1:]]['label'].tolist().pop())

PRT
SUFF
PRT
SUFF
PRT
